# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [43]:
#!pip3 install transformers

In [190]:
# import libraries
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
#from xgboost import XGBClassifier
#from transformers import GensimWord2VecVectorizer
#from transformers.gensim_word2vec import GensimWord2VecVectorizer
from sklearn.linear_model import LogisticRegression
#from gensim.models import word2vec
from gensim.models import Word2Vec
from sklearn.svm import LinearSVC
#from transformer import StartingVerbExtractor
from sklearn.decomposition import TruncatedSVD
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier

In [191]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [192]:
# load data from database
def load_data(database_filepath):
    #New.db is database_filepath, New is table name
    engine = create_engine('sqlite:///' + database_filepath)
    #table_name =  
    df = pd.read_sql_table('New',engine)
    X = df.message.values
    y = df.iloc[:, 4:]
    print(X.shape,'\n',y.columns,'\n',y.shape)
    return X, y


In [213]:
#df.message

In [156]:
load_data('New.db')

(26215,) 
 Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object') 
 (26215, 36)


(array(['Weather update - a cold front from Cuba that could pass over Haiti',
        'Is the Hurricane over or is it not over',
        'Looking for someone but no name', ...,
        "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families.",
        'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.',
        'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.'], dtype=object),
        related  request  offer  aid_related  medical_help  medical_products  \
 0            1        0      0            0             0                 0   
 1            1        0      0            1             0            

In [48]:
for col in df.columns[4:]:
    print(col,'\n',df[col].value_counts(),'\n')


related 
 1    20093
0     6122
Name: related, dtype: int64 

request 
 0    21741
1     4474
Name: request, dtype: int64 

offer 
 0    26097
1      118
Name: offer, dtype: int64 

aid_related 
 0    15355
1    10860
Name: aid_related, dtype: int64 

medical_help 
 0    24131
1     2084
Name: medical_help, dtype: int64 

medical_products 
 0    24902
1     1313
Name: medical_products, dtype: int64 

search_and_rescue 
 0    25491
1      724
Name: search_and_rescue, dtype: int64 

security 
 0    25744
1      471
Name: security, dtype: int64 

military 
 0    25355
1      860
Name: military, dtype: int64 

child_alone 
 0    26215
Name: child_alone, dtype: int64 

water 
 0    24543
1     1672
Name: water, dtype: int64 

food 
 0    23292
1     2923
Name: food, dtype: int64 

shelter 
 0    23901
1     2314
Name: shelter, dtype: int64 

clothing 
 0    25810
1      405
Name: clothing, dtype: int64 

money 
 0    25611
1      604
Name: money, dtype: int64 

missing_people 
 0    25917
1

In [49]:
 df.columns

Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

### 2. Write a tokenization function to process your text data

In [193]:
def tokenize(message):
    tokens = word_tokenize(message)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

In [194]:
# Build a custom transformer which will extract the starting verb of a sentence
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    Starting Verb Extractor class
    
    This class extract the starting verb of a sentence,
    creating a new feature for the ML classifier
    """

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    # Given it is a tranformer we can return the self 
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [195]:
def build_model():
    pipeline = Pipeline([
        ('vect',CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    return pipeline


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [196]:
X_train,X_test,y_train,y_test = train_test_split(X, y)

In [197]:
model = build_model()

In [198]:
model.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [199]:
def evaluate_model(model, X_test, y_test, category_names):
    y_pred = model.predict(X_test)
    y_pred = pd.DataFrame(y_pred)
    category_names = y.columns
    category_names
    print(classification_report(y_test, y_pred, target_names=category_names))

In [200]:
print('Evaluating model...')
evaluate_model(model, X_test, y_test, category_names)

Evaluating model...
                        precision    recall  f1-score   support

               related       0.82      0.92      0.87      5048
               request       0.82      0.39      0.53      1114
                 offer       0.00      0.00      0.00        18
           aid_related       0.76      0.50      0.60      2722
          medical_help       0.69      0.07      0.13       514
      medical_products       0.62      0.05      0.09       315
     search_and_rescue       0.73      0.06      0.11       178
              security       0.00      0.00      0.00       109
              military       0.56      0.03      0.05       194
           child_alone       0.00      0.00      0.00         0
                 water       0.85      0.19      0.30       427
                  food       0.85      0.31      0.45       763
               shelter       0.82      0.18      0.30       604
              clothing       0.75      0.08      0.15       112
                 mo

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [54]:
#from sklearn.utils.multiclass import type_of_target
#type_of_target(y_test)
#type_of_target(y_pred)
## due to Unknown label type

In [165]:
print(type(y_pred),type(y_test),type(X_test))

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> <class 'numpy.ndarray'>


In [214]:
#category_names = y.columns
#category_names

In [57]:
#print(classification_report(y_test,y_pred,target_names=category_names)) ## might we have 2?! replace with 1 .. yes "lazm tb2a 0 or 1 bs"

                        precision    recall  f1-score   support

               related       0.83      0.93      0.88      5042
               request       0.84      0.37      0.52      1095
                 offer       0.00      0.00      0.00        35
           aid_related       0.75      0.51      0.61      2670
          medical_help       0.68      0.06      0.11       516
      medical_products       0.63      0.08      0.14       298
     search_and_rescue       0.67      0.03      0.06       185
              security       0.00      0.00      0.00       104
              military       0.68      0.07      0.13       216
           child_alone       0.00      0.00      0.00         0
                 water       0.85      0.26      0.39       387
                  food       0.81      0.36      0.50       692
               shelter       0.80      0.19      0.30       552
              clothing       0.70      0.13      0.22       105
                 money       0.67      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [58]:
"""
cat_names = list(df.columns[4:])

for i in range(len(cat_names)):
    print("Category:", cat_names[i],"\n", classification_report(y_test.iloc[:, i].values, y_pred[:, i]))
    print('Accuracy of %25s: %.2f' %(cat_names[i], accuracy_score(y_test.iloc[:, i].values, y_pred[:,i])))
    
"""

'\ncat_names = list(df.columns[4:])\n\nfor i in range(len(cat_names)):\n    print("Category:", cat_names[i],"\n", classification_report(y_test.iloc[:, i].values, y_pred[:, i]))\n    print(\'Accuracy of %25s: %.2f\' %(cat_names[i], accuracy_score(y_test.iloc[:, i].values, y_pred[:,i])))\n    \n'

In [20]:
#X_test = pd.DataFrame(X_test)

### 6. Improve your model
Use grid search to find better parameters. 

In [204]:
def build_model_CV():
    pipeline = Pipeline([
        ('vect',CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

    parameters = {
        'clf__estimator__min_samples_split': [2, 4],
        'clf__estimator__criterion': ['entropy', 'gini'],
    } 

    cv = GridSearchCV(pipeline, parameters,n_jobs=4, verbose=2)
    
    return cv

In [205]:
model_cv = build_model_CV()
model_cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__min_samples_split': [2, 4], 'clf__estimator__criterion': ['entropy', 'gini']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [208]:
model_cv.best_params_

{'clf__estimator__criterion': 'gini', 'clf__estimator__min_samples_split': 2}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [215]:
#cv.fit(X_train, y_train)

In [24]:
#y_pred_cv = cv.predict(X_test)

In [216]:
#print(classification_report(y_test,y_pred_cv,target_names=category_names))

In [207]:
model = model_cv
evaluate_model(model, X_test, y_test, category_names)

                        precision    recall  f1-score   support

               related       0.82      0.93      0.87      5048
               request       0.81      0.37      0.50      1114
                 offer       0.00      0.00      0.00        18
           aid_related       0.74      0.51      0.60      2722
          medical_help       0.51      0.06      0.11       514
      medical_products       0.64      0.09      0.16       315
     search_and_rescue       0.50      0.02      0.04       178
              security       1.00      0.01      0.02       109
              military       0.81      0.07      0.12       194
           child_alone       0.00      0.00      0.00         0
                 water       0.82      0.11      0.20       427
                  food       0.83      0.37      0.51       763
               shelter       0.85      0.21      0.34       604
              clothing       0.89      0.07      0.13       112
                 money       0.86      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [113]:
"""
pipeline2 = Pipeline([
    ('features',FeatureUnion([
    
    ('text_pipeline',Pipeline([
        ('vect',CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer())
      ])),
        
      ('best', TruncatedSVD())
    ])),
    
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])
    
"""

In [209]:
from sklearn.tree import DecisionTreeClassifier
pipeline2 = Pipeline([
    ('vect', CountVectorizer()),
    ('best', TruncatedSVD()),
    #('starting_verb', StartingVerbExtractor()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
])

In [69]:
pipeline2.get_params()

{'memory': None, 'steps': [('features', FeatureUnion(n_jobs=1,
          transformer_list=[('text_pipeline', Pipeline(memory=None,
        steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_... TruncatedSVD(algorithm='randomized', n_components=2, n_iter=5,
          random_state=None, tol=0.0))],
          transformer_weights=None)),
  ('clf',
   MultiOutputClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
             intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
             penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
             verbose=0, warm_start=False),
              n_jobs=1))], 'features': FeatureUnion(n_jobs=1,
        transformer_list=[('text_pipeline', Pipeline(memory=None,
      steps=[('vect',

In [210]:
pipeline2.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ion_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
           n_jobs=1))])

In [67]:
#This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0 when I used linear svc
#for col in y_train:
#    print(col,'\n',y_train[col].value_counts(),'\n')

related 
 1    15051
0     4610
Name: related, dtype: int64 

request 
 0    16282
1     3379
Name: request, dtype: int64 

offer 
 0    19578
1       83
Name: offer, dtype: int64 

aid_related 
 0    11471
1     8190
Name: aid_related, dtype: int64 

medical_help 
 0    18093
1     1568
Name: medical_help, dtype: int64 

medical_products 
 0    18646
1     1015
Name: medical_products, dtype: int64 

search_and_rescue 
 0    19122
1      539
Name: search_and_rescue, dtype: int64 

security 
 0    19294
1      367
Name: security, dtype: int64 

military 
 0    19017
1      644
Name: military, dtype: int64 

child_alone 
 0    19661
Name: child_alone, dtype: int64 

water 
 0    18376
1     1285
Name: water, dtype: int64 

food 
 0    17430
1     2231
Name: food, dtype: int64 

shelter 
 0    17899
1     1762
Name: shelter, dtype: int64 

clothing 
 0    19361
1      300
Name: clothing, dtype: int64 

money 
 0    19205
1      456
Name: money, dtype: int64 

missing_people 
 0    19435
1

In [211]:
y_pred2 = pipeline2.predict(X_test)
y_pred2 = pd.DataFrame(y_pred)

In [212]:
print(classification_report(y_test,y_pred2))

             precision    recall  f1-score   support

          0       0.78      0.86      0.82      5048
          1       0.16      0.07      0.10      1114
          2       0.00      0.00      0.00        18
          3       0.42      0.28      0.33      2722
          4       0.09      0.01      0.01       514
          5       0.03      0.00      0.01       315
          6       0.11      0.01      0.01       178
          7       0.00      0.00      0.00       109
          8       0.05      0.01      0.01       194
          9       0.00      0.00      0.00         0
         10       0.08      0.02      0.03       427
         11       0.13      0.05      0.07       763
         12       0.10      0.02      0.04       604
         13       0.05      0.01      0.02       112
         14       0.17      0.01      0.01       158
         15       0.00      0.00      0.00        64
         16       0.00      0.00      0.00       216
         17       0.03      0.00      0.01   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [ ]:
#pipeline2.best_params_

### 9. Export your model as a pickle file

In [ ]:
#def save_model(model, model_filepath):
    #model_filepath = Disaster_Response.pk
    #with open (model_filepath, 'wb') as f:
#    pickle.dump(model, f)
#print('Trained model is saved.')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.